# COGS 118B - Final Project

# Tetris AI Backtracking Comparison

## Group members

- Pierce Nguyen
- Yiyi Huang
- Xunzhi He
- Nathalie Franklin

# Abstract 
This section should be short and clearly stated. It should be a single paragraph <200 words.  It should summarize: 
- what your goal/problem is
- what the data used represents 
- the solution/what you did
- major results you came up with (mention how results are measured) 

__NB:__ this final project form is much more report-like than the proposal and the checkpoint. Think in terms of writing a paper with bits of code in the middle to make the plots/tables

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
In our project, we investigated various backtracking methods and heuristics to optimize piece placement to maximize score in a simplified Tetris environment. The data for this project is dynamically generated game states that include the current board configuration along with the falling piece. We use several backtracking approaches using different heuristics such as score, score+height, full heuristics, full heuristics + alpha‐beta pruning and compare their performance to a Deep Q‐Network (DQN) model. We find that backtracking methods with domain‐specific heuristics often outperform the DQN, indicating that forward‐search and tailored board‐evaluation metrics can lead to more effective strategies. These findings are measured primarily through the Tetris score, which reflects the number of cleared lines.

# Background

Tetris has long been seen as an NP‐hard problem<a name="demaine"></a><sup>[1]</sup>, making it a strong benchmark for AI research. Early strategies for Tetris often relied on handcrafted heuristics such as penalizing holes or high stacks to place pieces effectively. However, these heuristic approaches can struggle with looking further ahead, prompting the use of search methods such as backtracking and alpha‐beta pruning. Meanwhile, reinforcement learning, especially Deep Q‐Networks (DQN), has been applied to a range of complex control tasks, including some variants of Tetris<a name="mnih"></a><sup>[23]</sup>. Despite successes in other domains, DQN often faces challenges with Tetris’s sparse rewards and large state space, highlighting the continued importance of search‐based techniques. By comparing backtracking algorithms with and without alpha‐beta pruning to a DQN approach, we aim to clarify how heuristic search can exploit domain knowledge more efficiently than purely data‐driven methods.

<br> <a name="demainenote">[1]</a> Demaine, E. D., Hohenberger, S., & Liben‐Nowell, D. (2003). *Tetris is hard, even to approximate.* Computational Complexity, 13(4), 426–453. <a name="mnihnote">[2]</a> Mnih, V., Kavukcuoglu, K., Silver, D., Rusu, A. A., et al. (2015). *Human‐level control through deep reinforcement learning.* Nature, 518(7540), 529–533.

# Problem Statement

Clearly describe the problem that you are solving. Avoid ambiguous words. The problem described should be well defined and should have at least one ML-relevant potential solution. Additionally, describe the problem thoroughly such that it is clear that the problem is quantifiable (the problem can be expressed in mathematical or logical terms), measurable (the problem can be measured by some metric and clearly observed), and replicable (the problem can be reproduced and occurs more than once).

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------

The goal of this project is to develop an AI that optimally places pieces in a Tetris game to maximize the game score, defined as the total number of cleared lines.

This problem is quantifiable because the performance metric can be expressed based on the number of cleared rows and penalized by board features such as aggregate height, holes, and bumpiness. It is measurable since each game run produces a score and a number of moves, which serve as clear indicators of performance. Moreover, the problem is replicable because the game environment can be reset to a predefined state, and the same sequence of tetrominoes can be used across different trials.

Our approach compares machine learning methods—specifically Deep Q-Networks (DQN)—with search-based methods (backtracking, with and without alpha-beta pruning) that use domain-specific heuristics. This comparison addresses the challenges of sparse rewards in DQN and evaluates whether heuristic search can more consistently yield higher scores in this NP-hard problem.

# Data

Since our problem is a live game, we don't have a huge dataset that we feed into the model to train it. Instead we will feed the current board state of the tetris game into our algorithm and have it pick the best moves and update the board state accordingly. Some critical variables would be the tetris board which we plan to represent as an array, the tetris pieces which we also plan to represent as an array. We also will need to account for all the possible rotations for the pieces. We will also need to make sure we update the board after each move for any cleared pieces.


# Proposed Solution

In this section, clearly describe a solution to the problem. The solution should be applicable to the project domain and appropriate for the dataset(s) or input(s) given. Provide enough detail (e.g., algorithmic description and/or theoretical properties) to convince us that your solution is applicable. Make sure to describe how the solution will be tested.  

If you know details already, describe how (e.g., library used, function calls) you plan to implement the solution in a way that is reproducible.

If it is appropriate to the problem statement, describe a benchmark model<a name="sota"></a>[<sup>[3]</sup>](#sotanote) against which your solution will be compared. 

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Our solution employs a dual approach to address optimal piece placement in Tetris. First, we implement a search-based method using a backtracking algorithm with fixed depth. In this method, a list of upcoming pieces is preloaded and, for each piece, all legal moves are generated using functions like `get_legal_placements` and `simulate_placement`. The algorithm recursively explores these moves up to a set depth, evaluating the resulting board states with a heuristic that combines the game score, aggregate height, number of holes, and bumpiness. Two variants of the backtracking approach are implemented: one using full heuristics without alpha-beta pruning and another incorporating alpha-beta pruning to efficiently cut off suboptimal branches. We also tested out much more basic heuristics such as just score and score + height to see how the heuristics impact the average score. The output for each variant is recorded in CSV files which log the game number, number of moves, and final score.

To compare to the backtracking approach, we utilize DQN for a benchmark model. For this, we modified an open source, pre-trained model found on GitHub<a name="nguyen"></a><sup>[1]</sup>,. The model is trained with three linear, fully connected layers, with ReLU activation. The states are defined by the number of lines cleared, number of holes, bumpiness, and height. The actions are defined as a tuple of the figure rotation and x position value. On each epoch, the algorithm will make predictions, using the current model, and either select the index with the best prediction or make a random choice with epsilon probability. The model was trained using the following hyperparameters: batch size = 512, gamma = 0.99, initial epsilon = 1, final epsilon = 1e-3, total epochs = 3000, and replay memory size = 30000. A `DQN` class (with methods for `init` and `forward`) is needed for this implementation; additionally, a `train` method is needed to running the model training process. 

<a name="nguyen">[1]</a> Nguyen, V. (2023). Deep Q-Network for Tetris [Computer software]. GitHub. https://github.com/vietnh1009/Tetris-deep-Q-learning-pytorch/

TODO: potentially add equations/snippets of code (mainly heuristics such as `evaluate_state`)

# Evaluation Metrics

We use the average final score for each  as the primary evaluation metric. For each algorithm, we run 100 games—each terminated after 150 moves. We did this because our full heuristics model did not lose even after running for an hour which would make running 100 games impossible. The average final score is calculated by summing the final scores from all games and then dividing by the total number of games. This metric directly quantifies how effectively an algorithm can clear lines and achieve high scores. The score is calculated according to Tetris' rules with lines cleared as the only contributing factor to the overall score.

Note: eval metrics are strictly in relation to how we compare our different algos not math behind backtracking/heuristics

# Results `TODO`: Regenerate box_plot.png, compare DQN and AB, table comparing all 4 backtracking algos (mean, median, std), table comparing dqn and ab (mean, median, std)
`TODO` Overall results are DQN wins by about 25%. Likely due to limited depth search (see Discussion for explanation). Then for backtracking obv full heuristics wins.


You may have done tons of work on this. Not all of it belongs here. 
Reports should have a __narrative__. Once you've looked through all your results over the quarter, decide on one main point and 2-4 secondary points you want us to understand. Include the detailed code and analysis results of those points only; you should spend more time/code/plots on your main point than the others.

If you went down any blind alleys that you later decided to not pursue, please don't abuse the TAs time by throwing in 81 lines of code and 4 plots related to something you actually abandoned.  Consider deleting things that are not important to your narrative.  If its slightly relevant to the narrative or you just want us to know you tried something, you could keep it in by summarizing the result in this report in a sentence or two, moving the actual analysis to another file in your repo, and providing us a link to that file.

### Comparing Various Backtracking Algorithms and Heuristics

![Graph Title](grpah\box_plot1.png)


| Experiment                     | Mean  | Median | Standard Deviation |
|--------------------------------|------:|-------:|-------------------:|
| Score                          |  0.99 |    0.0 |            1.648354 |
| Score + Height                 | 56.43 |   65.5 |           23.449604 |
| Full Heuristics                | 77.12 |   76.0 |            7.317172 |
| Full Heuristics + AB Pruning    | 76.68 |   76.5 |            7.428963 |


To analyze different heuristics for our backtracking model, we implemented and compared four different models. The Score model utilizes the game score as a heuristic; the Score+Height model utilizes both game score and height; the Full Heuristics model utilizes game score, height, bumpiness, and number of holes. The Full Heuristics + AB Pruning (also aliased as just AB Pruning) model utilizes the same heuristics as the Full Heuristics model with the additional use of Alpha Beta pruning. All backtracking models used a depth of 3.

Producing the worst results, to no surprise, is the Score model (median = 0) since it uses the least heuristics compared to the other models. The next model with the lowest median (65.5) is the Score+Height model. Although this model has a large difference in median score compared to the Score model, it is far outperformed by both Full Heuristics and AB Pruning. 

The results of Full Heuristics and Full Heuristics + AB Pruning both have similar means and medians with both metrics differing in less than one score point. The standard deviation of both also differ by ~0.1. 

In analyzing the ranges of scores for each, the Score model leads to the smallest standard deviation, since few scores are far from zero. The Score+Height model lead with the largest standard deviation (23.45), while Full Heuristics and AB Pruning consistently perform well; their standard deviations are 7.32 and 7.43, respectively.

### Comparing Backtracking vs DQN
`TODO:` Fill this out
Note: only compare using the best backtracking algo not the worse 3

![Graph Title](grpah\box_plot2.png)

| Experiment   | Mean   | Median | Standard Deviation |
|-------------|--------|--------|--------------------|
| DQN         | 108.57 | 110.0  | 15.411694         |
| AB Pruning  | 76.68  | 76.5   | 7.428963          |


-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Note: add more subsections if you think we need them but idk what other results we have


# Discussion

### Interpreting the result
`TODO`
OK, you've given us quite a bit of tech informaiton above, now its time to tell us what to pay attention to in all that.  Think clearly about your results, decide on one main point and 2-4 secondary points you want us to understand. Highlight HOW your results support those points.  You probably want 2-5 sentences per point.

The results clearly show that the DQN approach, with a mean score of 108.57 and a median of 110, significantly outperformed the best backtracking variant (Full Heuristics + AB Pruning) which had a mean of 76.68 and a median of 76.5. These results indicate that the reinforcement learning method is better at navigating the complex decision space in Tetris. The higher overall scores achieved by DQN suggest that its ability to learn from past experiences and optimize decisions over time gives it an edge over the search-based backtracking approach. Despite the limitations imposed by shallow search depth in backtracking, DQN’s performance is consistently superior.

Due to time and resource constraints, the backtracking algorithm was limited to a search depth of 3 moves. The main challenge of using backtracking for Tetris is the exponential growth in computational needs as the depth is increased due to the high number of possible board states at each step. Using a shallow depth likely prevented backtracking from fully exploiting the potential benefits of its various heuristics. As a result, even when all the heuristics were applied, the performance remained significantly below that of the DQN approach. In theory, a deeper search depth would capture more states and long-term strategic moves, but the exponential growth in possibilities forces a trade-off between search depth and computational feasibility, limiting its overall effectiveness compared to learning-based methods such as DQN.

The addition of alpha-beta pruning to the full heuristics approach resulted in only a marginal decrease in the mean score (from 77.12 to 76.68) with a nearly identical median and standard deviation. This suggests that the pruning mechanism had limited benefit when the search space is already shallow due to the depth constraint. In scenarios with deeper search depths, alpha-beta pruning might prove more advantageous, but under the current limitations, its impact is minimal. 

While DQN achieves higher overall scores, it also exhibits a higher standard deviation (15.41) compared to the backtracking method with AB pruning (7.43). This increased variability may reflect the inherent randomness of the reinforcement learning approach, where the balance between exploration and exploitation sometimes leads to riskier, less consistent decisions. Unlike backtracking, which evaluates all possible moves within its search depth, DQN relies on learned approximations and as a result is more sensitive to changes in the game environment, leading to decisions that can vary widely and produce a higher standard deviation in performance. Despite this, the consistently higher mean and median scores underline DQN’s overall effectiveness, demonstrating that its performance gains outweigh the fluctuations.

### Limitations

Are there any problems with the work?  For instance would more data change the nature of the problem? Would it be good to explore more hyperparams than you had time for?   

One of our major limitations was computing power. Due to limited resources we could not explore a depth lower than 3 since it would take an extremely long time so we couldn't use alpha beta pruning to its full extent. With an increased depth we likely would have had a higher average score for our backtracking section. Another major limitation would be that we used a simplified version of tetris that doesn't have time scaling, the full block set, or as advanced piece movement (sliding pieces under each other etc). We used this simplified version to make it easier to integrate our algorithms into the tetris game.

### Future work
In the future, we want to improve our backtracking model by increasing the search depth, possibly using beam search to reduce computation while keeping better placements. We also think combining DQN with heuristics could create a stronger AI by learning better weights for board evaluations. Another idea is improving DQN’s state representation, maybe using CNNs to analyze board patterns instead of just hand-picked features. Trying other reinforcement learning methods like PPO or MCTS could also help. Lastly, we’d like to test our AI on a full Tetris version with more mechanics like T-spins and hold pieces to see if our strategies still hold up.

### Ethics & Privacy

While a Tetris AI seems pretty simple and narrow in scope it can have some ethical implications. Tetris is mostly a singleplayer game but tournaments and leaderboards do exist so its possible for someone to cheat using this AI to gain an unfair advantage. Looking through Deon's ethical checklist, since we aren't utilizing any dataset we don't need to worry about any biases or gaps in our data. Our code is unlikely to be used for a different purpose than playing Tetris but even if it is it will only be able to be used to predict where the best place to place a block on a gameboard is so there is a low potential for misuse/harm.

### Conclusion
`TODO`

Reiterate your main point and in just a few sentences tell us how your results support it. Mention how this work would fit in the background/context of other work in this field if you can. Suggest directions for future work if you want to.

<br> <a name="demainenote">[1]</a> Demaine, E. D., Hohenberger, S., & Liben‐Nowell, D. (2003). *Tetris is hard, even to approximate.* Computational Complexity, 13(4), 426–453. <a name="mnihnote">[2]</a> Mnih, V., Kavukcuoglu, K., Silver, D., Rusu, A. A., et al. (2015). *Human‐level control through deep reinforcement learning.* Nature, 518(7540), 529–533.